# **Reconnaissance de visages de la base donnée**

In [ ]:
!pip install face_recognition

     |████████████████████████████████| 100.2MB 42kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=0531f98cfcf06362bf95fbbdfa337ae716fe0d6f5995bd14c1724aa25beb1dd8
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [ ]:
import face_recognition
import numpy as np
import shutil
import os
import smtplib
import imghdr
import cv2
import time
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

In [ ]:
known_face_encodings = []
known_face_names = []
i=0
directory = r'/content/opencv/known_faces'
for filename in os.listdir(directory):
  image = face_recognition.load_image_file(os.path.join(directory, filename))
  face_encoding = face_recognition.face_encodings(image)[0]
  known_face_encodings.append(face_encoding)
  known_face_names.append(filename[:-4])
  print("reconnaissance terminée avec succès pour "+filename[:-4])

reconnaissance terminée avec succès pour achraf
reconnaissance terminée avec succès pour ahmed
reconnaissance terminée avec succès pour nadine
reconnaissance terminée avec succès pour jhon
reconnaissance terminée avec succès pour maria


# **Définiton de la fonction autorisation**

In [ ]:
def autorisation():
  test = 0
  dir = r'/content/opencv/testing'
  for file in os.listdir(dir):
    # Load an image with an unknown face
    unknown_image = face_recognition.load_image_file(os.path.join(dir, file))
    name = "unknown"
    # Find all the faces and face encodings in the unknown image
    face_locations = face_recognition.face_locations(unknown_image)
    face_encodings = face_recognition.face_encodings(unknown_image, face_locations)
    os.remove(os.path.join(dir, file))
    # Loop through each face found in the unknown image
    for face_encoding in face_encodings:
      # See if the face is a match for the known face(s)
      matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
      # Or instead, use the known face with the smallest distance to the new face
      face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
      best_match_index = np.argmin(face_distances)
      if matches[best_match_index]:
        name = known_face_names[best_match_index]
        test = 1
        break
  return name
    

# **Définition de la fonction send_known**

---



In [ ]:
def send_known(name):
  from email.message import EmailMessage
  content =  name + " a tenté d'obtenir l'accès pour entrer."
  EMAIL_ADDRESS = 'computervison.enis@gmail.com'
  EMAIL_PASSWORD = 'projetopencv2020'

  contacts = ['achraf.tarifa@enis.tn']
  for i in contacts : 
    msg = EmailMessage()
    msg['Subject'] = "TENTATIVE D'ACCES"
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = i
    msg.set_content(content)
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
      smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
      smtp.send_message(msg)

# **Définition de la fonction send_unknown**

In [ ]:
def send_unknown():
  fromaddr = 'computervison.enis@gmail.com'
  toaddr = 'achraf.tarifa@enis.tn'

  msg = MIMEMultipart()

  msg['From'] = fromaddr
  msg['To'] = toaddr
  msg['Subject'] = "ALERTE!!"

  body = "Une personne inconnue a tenté d'obtenir l'accès pour entrer."
  dire = r'/content/opencv/unknown'
  msg.attach(MIMEText(body, 'plain'))
  for file in os.listdir(dire):
    filename = file
    attachment = open(os.path.join(dire, file),"rb")
    part = MIMEBase('application', 'octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', "attachment; filename= %s" % filename)
    msg.attach(part)
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(fromaddr, 'projetopencv2020')
    text = msg.as_string()
    server.sendmail(fromaddr, toaddr, text)
    os.remove(os.path.join(dire, file))
  server.quit()

# **Lecture video ou live cam et détection des visages**

In [ ]:
#cap = cv2.VideoCapture(0) for live cam
cap = cv2.VideoCapture('/content/opencv/test2.3gpp')
t_end = time.time()
face_cascade = cv2.CascadeClassifier('/content/opencv/haarcascade_frontalface_default.xml')
count = 0

In [ ]:
while True:
    _, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    if faces != () :
      for (x, y, w, h) in faces:
        #cv2.imwrite("/content/opencv/ff/fr%d.jpg" % count, img)
        if(time.time()>=t_end):
            cv2.imwrite("/content/opencv/testing/frame%d.jpg" % count, img)
            count = count + 1
            nom = autorisation()
            if nom!="unknown":
              send_known(nom)
              print(nom)
              count = 0
              t_end = time.time()+60
            else :
              print(nom)
              if(count>15):
                cv2.imwrite("/content/opencv/unknown/frame%d.jpg" % count, img)
                send_unknown()
                count = 0
                t_end = time.time()+60

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """


unknown
unknown
unknown
unknown


KeyboardInterrupt: ignored